# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import pandas as pd 
import numpy
import re
from sqlalchemy import create_engine
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline ,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer ,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier ,AdaBoostClassifier
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import classification_report
import pickle

In [25]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('data',engine)
X = df.iloc[:,1]
Y = df.iloc[:,4:40]

### 2. Write a tokenization function to process your text data

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stopWords=set(stopwords.words('english'))

def tokenize(text):
    text=text.lower()
    text = re.sub(r'[^a-zA-Z0-9]' ,' ',text)
    token=word_tokenize(text)
    lemmatizer= WordNetLemmatizer() 
    words=[]
    for word in token:
        if word not in stopWords:
            word=lemmatizer.lemmatize(word)
            words.append(word)
    
    return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)), 
        ('tfidf' , TfidfTransformer()),
        ('clv', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
Xtrain ,Xtest ,Ytrain , Ytest =train_test_split(X , Y , test_size=0.75)
pipeline.fit(Xtrain ,Ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Ypredict= pipeline.predict(Xtest)
for i in range(36):
    print('--------------------------------------------------')
    print(Ytest.columns[i],':')
    print(classification_report(Ytest.iloc[:,i],Ypredict[:,i],target_names=Ytest.columns[i]))
    print('--------------------------------------------------')

related 0 :
             precision    recall  f1-score   support

          r       0.60      0.37      0.46      4625
          e       0.83      0.91      0.87     14912
          l       0.15      0.45      0.23       125

avg / total       0.77      0.78      0.77     19662

request 1 :
             precision    recall  f1-score   support

          r       0.88      0.98      0.93     16298
          e       0.81      0.37      0.51      3364

avg / total       0.87      0.88      0.86     19662

offer 2 :
             precision    recall  f1-score   support

          o       1.00      1.00      1.00     19570
          f       0.00      0.00      0.00        92

avg / total       0.99      1.00      0.99     19662

aid_related 3 :
             precision    recall  f1-score   support

          a       0.73      0.86      0.79     11510
          i       0.75      0.56      0.64      8152

avg / total       0.74      0.74      0.73     19662

medical_help 4 :
             precisi

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 11
  .format(len(labels), len(target_names

             precision    recall  f1-score   support

          c       0.98      1.00      0.99     19263
          o       0.55      0.02      0.03       399

avg / total       0.97      0.98      0.97     19662

other_weather 34 :
             precision    recall  f1-score   support

          o       0.95      1.00      0.97     18623
          t       0.59      0.02      0.04      1039

avg / total       0.93      0.95      0.92     19662

direct_report 35 :
             precision    recall  f1-score   support

          d       0.85      0.98      0.91     15841
          i       0.74      0.28      0.41      3821

avg / total       0.83      0.84      0.81     19662



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 13
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clv', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clv__estimator__bootstrap', 'clv__estimator__class_weight', 'clv__estimator__criterion', 'clv__estimator__max_depth', 'clv__estimator__max_features', 'clv__estimator__max_leaf_nodes', 'clv__estimator__min_impurity_decrease', 'clv__estimator__min_impurity_split', 'clv__estimator__min_samples_leaf', 'clv__estimator__min_samples_split', 'clv__estimator__min_weight_fraction_leaf', 'clv__estimator__n_estimators', 'clv__estimator__n_jobs', 'clv__estimator__oob_score', 'clv__estimator__random_state', 'clv__estimator__verbose', 'clv__estimator__

In [15]:
parameters = {
   
    'clv__estimator__max_features': ['auto', 'sqrt'],
    'clv__estimator__min_samples_leaf':[2,4,6],
    'clv__estimator__min_samples_split':[3,6,15],
}


cv = GridSearchCV(pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
cv.fit(Xtrain,Ytrain)
Ypredict= cv.predict(Xtest)
for i in range(36):
    print('--------------------------------------------------')
    print(Ytest.columns[i],':')
    print(classification_report(Ytest.iloc[:,i],Ypredict[:,i],target_names=Ytest.columns[i]))
    print('--------------------------------------------------')

--------------------------------------------------
related :
             precision    recall  f1-score   support

          r       0.65      0.35      0.45      4625
          e       0.82      0.94      0.88     14912
          l       0.64      0.11      0.19       125

avg / total       0.78      0.80      0.77     19662

--------------------------------------------------
--------------------------------------------------
request :
             precision    recall  f1-score   support

          r       0.87      0.99      0.93     16298
          e       0.83      0.30      0.44      3364

avg / total       0.86      0.87      0.84     19662

--------------------------------------------------
--------------------------------------------------
offer :
             precision    recall  f1-score   support

          o       1.00      1.00      1.00     19570
          f       0.00      0.00      0.00        92

avg / total       0.99      1.00      0.99     19662

-------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 11
  .format(len(labels), len(target_names


--------------------------------------------------
--------------------------------------------------
direct_report :
             precision    recall  f1-score   support

          d       0.84      0.98      0.91     15841
          i       0.76      0.24      0.37      3821

avg / total       0.83      0.84      0.80     19662

--------------------------------------------------


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)), 
        ('tfidf' , TfidfTransformer()),
        ('clv', MultiOutputClassifier(AdaBoostClassifier()))
])
pipeline.fit(Xtrain ,Ytrain)
Ypredict= cv.predict(Xtest)
for i in range(36):
    print('--------------------------------------------------')
    print(Ytest.columns[i],':')
    print(classification_report(Ytest.iloc[:,i],Ypredict[:,i],target_names=Ytest.columns[i]))
    print('--------------------------------------------------')


--------------------------------------------------
related :
             precision    recall  f1-score   support

          r       0.65      0.35      0.45      4625
          e       0.82      0.94      0.88     14912
          l       0.64      0.11      0.19       125

avg / total       0.78      0.80      0.77     19662

--------------------------------------------------
--------------------------------------------------
request :
             precision    recall  f1-score   support

          r       0.87      0.99      0.93     16298
          e       0.83      0.30      0.44      3364

avg / total       0.86      0.87      0.84     19662

--------------------------------------------------
--------------------------------------------------
offer :
             precision    recall  f1-score   support

          o       1.00      1.00      1.00     19570
          f       0.00      0.00      0.00        92

avg / total       0.99      1.00      0.99     19662

-------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 11
  .format(len(labels), len(target_names

             precision    recall  f1-score   support

          c       0.98      1.00      0.99     19263
          o       0.00      0.00      0.00       399

avg / total       0.96      0.98      0.97     19662

--------------------------------------------------
--------------------------------------------------
other_weather :
             precision    recall  f1-score   support

          o       0.95      1.00      0.97     18623
          t       0.66      0.02      0.04      1039

avg / total       0.93      0.95      0.92     19662

--------------------------------------------------
--------------------------------------------------
direct_report :
             precision    recall  f1-score   support

          d       0.84      0.98      0.91     15841
          i       0.76      0.24      0.37      3821

avg / total       0.83      0.84      0.80     19662

--------------------------------------------------


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 13
  .format(len(labels), len(target_names))


### 9. Export your model as a pickle file

In [23]:
pickle.dump(cv ,open('model.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.